# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
pip install cartopy


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install geoviews

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import cartopy
import geoviews
import pyproj
# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tsiombe,-25.3000,45.4833,20.60,82,87,1.90,MG,1698277662
1,1,hawaiian paradise park,19.5933,-154.9731,27.80,81,75,3.09,US,1698277593
2,2,kingscliff,-28.2667,153.5667,26.52,65,100,4.12,AU,1698277663
3,3,adamstown,-25.0660,-130.1015,22.83,81,23,6.30,PN,1698277663
4,4,papatowai,-46.5619,169.4708,7.39,91,81,7.09,NZ,1698277663


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)



# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df= city_data_df.loc[(city_data_df["Max Temp"]<26)&
                                    (city_data_df["Max Temp"]>22)&
                                    (city_data_df["Humidity"]>20)&
                                    (city_data_df["Humidity"]<65)&
                                    (city_data_df["Cloudiness"]<20),:
                                   
                                  ]

# Drop any rows with null values
ideal_weather_df=ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
46,46,lichinga,-13.3128,35.2406,22.03,45,0,2.81,MZ,1698277673
193,193,nouadhibou,20.9310,-17.0347,25.99,53,0,8.75,MR,1698277711
196,196,riverside,33.7333,-116.0508,24.44,35,0,1.54,US,1698277607
230,230,crane,31.3974,-102.3501,23.15,43,0,0.99,US,1698277721
299,299,selma,36.5708,-119.6121,25.96,49,0,2.57,US,1698277739
345,345,al kharijah,25.4514,30.5464,25.08,34,0,2.75,EG,1698277752
370,370,malango,-2.9627,119.9001,24.62,59,17,0.68,ID,1698277758
575,575,al jawf,29.5000,38.7500,22.05,41,0,4.07,SA,1698277825


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""
# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
46,lichinga,MZ,-13.3128,35.2406,45,
193,nouadhibou,MR,20.9310,-17.0347,53,
196,riverside,US,33.7333,-116.0508,35,
230,crane,US,31.3974,-102.3501,43,
299,selma,US,36.5708,-119.6121,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    print(name_address.status_code)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
200
lichinga - nearest hotel: Residencial 2+1
200
nouadhibou - nearest hotel: Hotel valencia
200
riverside - nearest hotel: No hotel found
200
crane - nearest hotel: No hotel found
200
selma - nearest hotel: Best Western
200
al kharijah - nearest hotel: Kharga Hotel
200
malango - nearest hotel: Madarana
200
al jawf - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
46,lichinga,MZ,-13.3128,35.2406,45,Residencial 2+1
193,nouadhibou,MR,20.9310,-17.0347,53,Hotel valencia
196,riverside,US,33.7333,-116.0508,35,No hotel found
230,crane,US,31.3974,-102.3501,43,No hotel found
299,selma,US,36.5708,-119.6121,49,Best Western
345,al kharijah,EG,25.4514,30.5464,34,Kharga Hotel
370,malango,ID,-2.9627,119.9001,59,Madarana
575,al jawf,SA,29.5000,38.7500,41,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)



# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)